# CmapaignReadRateAnalysis

## Dependance

In [11]:
import sys,os
import datetime, time
import cx_Oracle
import pandas as pd
import numpy as np
import traceback
import logging

BASE_DIR = r"C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test"
sys.path.append(BASE_DIR)

from string import Template

from core import sdm_check
from core.configadmin import ConfigAdmin
from core.color_me import ColorMe
from core.outlook import send_email
from core import log_ctrl
from core.camp_res import campaign_res

In [12]:
# TODO:Config
config_path = os.path.join(BASE_DIR, 'conf')
config_name = 'setting.ini'
conf = ConfigAdmin(config_path, config_name)
# Template
D3_excel = os.path.join(conf.read_config('Cofig_Path','D3_Report'),f'XRM Campaign Report {time.strftime("%Y%m%d",time.localtime())}.xlsx')
D3_html_R = os.path.join(conf.read_config('Cofig_Path','D3_Report'),f'XRM Regular Campaign Report {time.strftime("%Y%m%d",time.localtime())}.html')
D3_html_O = os.path.join(conf.read_config('Cofig_Path','D3_Report'),f'XRM One Time Campaign Report {time.strftime("%Y%m%d",time.localtime())}.html')
MyTemp = os.path.join (config_path, 'CSS', conf.read_config('CSS_Style', 'XRMCampaignTemplate'))
MyCss_Style = os.path.join (config_path, 'CSS', conf.read_config('CSS_Style', 'table_style'))
# Log Info
log_info = log_ctrl.de8ug_log(logger_name='CampaignReadRateReprot', log_file=os.path.join(BASE_DIR, 'log', f'{time.strftime("%Y%m",time.localtime())}-monitor.log'), level=logging.INFO)
# SQL Script
CampaignListWaitToCheck = os.path.join(BASE_DIR, 'db','sql','CampaignListWaitToCheck.sql')
# CampaignAnaylysis = os.path.join(BASE_DIR, 'db','sql','CampaignAnaylysis.sql')
CampaignAnaylysis = os.path.join(BASE_DIR, 'db','sql','CampaignAnaylysis1.sql')
# SAS DB
user = conf.read_config("SAS_db", "user")
password = conf.read_config("SAS_db", "password")
service = conf.read_config("SAS_db", "service")
# panda option
a = lambda x: format(x, '.2%') if x != np.nan else ""
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
# pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
# pd.set_option('max_colwidth',100)
# pd.set_option("display.max_columns",18)
pd.set_option('display.width', 2000)    # 设置行宽
pd.set_option('colheader_justify', 'center')    #设置居中

## Scrips

### GestList
1. 获取任务列表
2. 计算Channel Response

#### 获取任务列表

In [19]:
db = cx_Oracle.connect(user, password, service)
con = db.cursor()
with open(CampaignListWaitToCheck, 'r') as f:
#     print(f.read())
    sql = Template(f.read()).substitute(LaunchDays=11, EndDays=1)
    row = con.execute(sql).fetchall()
    cols = [i[0] for i in con.description]

In [22]:
df = pd.DataFrame(data=row, index=[i[0] for i in row], columns=cols)
# ChannlList = [k for k,v in dict(df.index).items()]
# df

In [23]:
SubjectTypeList = list(df.drop_duplicates(['SUBJECT_TYPE_NM'])['SUBJECT_TYPE_NM'])
CheckModelList = list(df.drop_duplicates(['CHECK_MODEL'])['CHECK_MODEL'])

#### Emp 计算

In [25]:
# EMPLOYEE
a = "'"
for i in df[df.SUBJECT_TYPE_NM=='Employee']['CELL_PACKAGE_SK']:
    a = a + i.replace(' ',"','") + "','"
TB1 = 'CI_CONTACT_HISTORY_EMP'
TB2 = 'CI_RESPONSE_HISTORY_EMP'
SK = a[:-2]

with open(CampaignAnaylysis, 'r') as f:
    print()
    sql = Template(f.read()).substitute(TB1=TB1, TB2=TB2, SK=SK, RESDY=2)
row1 = con.execute(sql).fetchall()
cols1 = [i[0] for i in con.description]

In [26]:
df1 = pd.DataFrame(data=row1, index=[i[1] for i in row1], columns=cols1)
# ChannlList = [k for k,v in dict(df.index).items()]
df1

,LAUNCH_DATE,CAMPAIGN_CD,CAMPAIGN_NM,CAMP_FREQUENCY,CHANNEL_CD,ID_TEMPLATE,NAME_SALES_LOCATION_PROVINCE,COMMUNICATION_CD,CELL_PACKAGE_SK,RESPONSE_TRACKING_CD,CAMP_OWNER,CAMP_BUSINESS_TARGET,CAMP_TYPE,TARGET,CAMP_BUSINESS_TYPE,CAMP_REQ_TICKET_NUMBER,PRIMARY_EXPORT_COUNT,TOTAL_DELIVERY,DELIVERY_OK,DELIVERY_FAILED,CNT_REPLY,CNT_CONTACT_SUCCESS,CNT_CONTACT_FAILED,INBOX_READ,POPUP_READ,PUSH_READ,RED_PACKET_SUCCESS,RED_PACKET_FAILED,URL_CLICK,CNT_REMOVE
CAMP1982,2020-07-20 17:40:58,CAMP1982,SMT_Lucky_A_DSM_SCR-1524,One Time,SMT,12939,None,COMM10150,36238,362380101,Jason Ji,Sales uplift,POS Loan,DSM,Sales Network,SCR-1524,3815,3815,3815,0,0,0,0,297,524,0,0,0,0,0
CAMP1987,2020-07-20 17:42:40,CAMP1987,SMT_Lucky_B_DSM_SCR-1526,One Time,SMT,13145,None,COMM10163,36239,362390101,Jason Ji,Sales uplift,POS Loan,DSM,Sales Network,SCR-1526,3626,3626,3626,0,0,0,0,254,444,0,0,0,0,0
CAMP1997,2020-07-21 16:51:50,CAMP1997,SMT_SAMT&SA_Lucky_SAMT_SCR-1529,One Time,SMT,13143,None,COMM10188,36393,363930101,Jason Ji,Sales uplift,POS Loan,DSM,Sales Network,SCR-1529,1384,1384,1384,0,0,0,0,2,5,0,0,0,0,0
CAMP1996,2020-07-21 17:10:09,CAMP1996,SMT_Extra_monthly_bonus_CM_SCR-1530,One Time,SMT,13144,None,COMM10183,36401,364010101,Jessie Wang,Sales uplift,POS Loan,DSM,Sales Network,SCR-1530,215,215,215,0,0,0,0,61,93,0,0,0,0,0
CAMP1998,2020-07-21 17:11:48,CAMP1998,SMT_Final_Bonus _Coefficient_Adjustment_DSM_SC...,One Time,SMT,13141,None,COMM10193,36402,364020101,Jessie Wang,Sales uplift,POS Loan,DSM,Sales Network,SCR-1532,1265,1265,1265,0,0,0,0,807,1040,0,0,0,0,0
CAMP2000,2020-07-22 17:41:58,CAMP2000,SMT_Summer_Scholarships_Notice_SCR-1448,One Time,SMT,12347,None,COMM10208,36486,364860101,Jessie Wang,Service campaign,POS Loan,DSM,Sales Network,SCR-1448,9669,9669,9669,0,0,0,0,750,1177,0,0,0,0,0
CAMP1999,2020-07-23 13:45:09,CAMP1999,SMT_HR_Survey_DSM_SCR-1531,One Time,SMT,13142,None,COMM10200,36490,364900101,Jason Ji,Sales uplift,POS Loan,DSM,Sales Network,SCR-1531,9618,9618,9618,0,0,0,0,676,1095,0,0,0,201,0
CAMP2001,2020-07-23 16:17:39,CAMP2001,SMT_TIPPER_Unblock_DSM_SCR-1534,One Time,SMT,12331,None,COMM10232,36506,365060101,Jason Ji,Sales uplift,POS Loan,DSM,Sales Network,SCR-1534,949,949,949,0,0,0,0,97,226,0,0,0,0,0
CAMP2002,2020-07-23 17:15:33,CAMP2002,SMT_Free_Helmet_TW_POS_SCR-1533,One Time,SMT,13140,None,COMM10235,36515,365150101,Jessie Wang,Sales uplift,POS Loan,DSM,Sales Network,SCR-1533,194,194,194,0,0,0,0,23,39,0,0,0,1,0
CAMP2003,2020-07-23 17:16:31,CAMP2003,SMT_Update_Notification_DSM_SCR-1542,One Time,SMT,13344,None,COMM10237,36516,365160101,Jessie Wang,Service campaign,POS Loan,DSM,Sales Network,SCR-1542,9618,9618,9618,0,0,0,0,633,1074,0,0,0,33,0


In [33]:
# TIPPER
db = cx_Oracle.connect(user, password, service)
con = db.cursor()
# a = "'"
# for i in df[df.SUBJECT_TYPE_NM=='Tipper']['CELL_PACKAGE_SK']:
#     a = a + i.replace(' ',"','") + "','"
TB1 = 'CI_CONTACT_HISTORY_TIP'
TB2 = 'CI_RESPONSE_HISTORY_TIP'
# SK = a[:-2]
SK = '34450,34569'
with open(CampaignAnaylysis, 'r') as f:
    sql = Template(f.read()).substitute(TB1=TB1, TB2=TB2, SK=SK, RESDY=7)
row2 = con.execute(sql).fetchall()
cols2 = [i[0] for i in con.description]
db.close()

In [34]:
df2 = pd.DataFrame(data=row2, index=[i[1] for i in row2], columns=cols2)
DataSet = df2[cols2[0:7] + cols2[10:14] + cols2[16:]]
# DataSet.columns
# By Temp Check
DataSetByTemp = DataSet.sort_values(by=['CHANNEL_CD','CAMPAIGN_NM','NAME_SALES_LOCATION_PROVINCE'])
# SMT By Data
SMTColumns = ['TOTAL_DELIVERY', 'DELIVERY_RATE', 'INBOX_READ', 'INBOX_RATE', 'POPUP_READ', 'POPUP_RATE', 'URL_CLICK', 'CLICK_RATE', 'PUSH_READ', 'PUSH_RATE']
SMTByDate = DataSetByTemp.loc[(DataSetByTemp['CHANNEL_CD'] == 'SMT')].copy()
SMTByDate['DELIVERY_RATE'] = SMTByDate.apply(lambda x: x['DELIVERY_OK']/x['DELIVERY_OK'], axis=1)
SMTByDate['INBOX_RATE'] = SMTByDate.apply(lambda x: x['INBOX_READ']/x['DELIVERY_OK'], axis=1)
SMTByDate['POPUP_RATE'] = SMTByDate.apply(lambda x: x['POPUP_READ']/x['DELIVERY_OK'], axis=1)
SMTByDate['CLICK_RATE'] = SMTByDate.apply(lambda x: x['URL_CLICK']/x['DELIVERY_OK'], axis=1)
SMTByDate['PUSH_RATE'] = SMTByDate.apply(lambda x: x['PUSH_READ']/x['DELIVERY_OK'], axis=1)
SMTByDate = SMTByDate.set_index(['NAME_SALES_LOCATION_PROVINCE'])
SMTByDate[['CAMPAIGN_NM', 'LAUNCH_DATE', 'DELIVERY_OK', 'DELIVERY_RATE', 'INBOX_READ', 'INBOX_RATE', 'POPUP_READ', 'POPUP_RATE', 'URL_CLICK', 'CLICK_RATE']].
# SMTByDate.to_html('test.html', columns=SMTColumns, float_format=a, border=1)

,CAMPAIGN_NM,LAUNCH_DATE,DELIVERY_OK,DELIVERY_RATE,INBOX_READ,INBOX_RATE,POPUP_READ,POPUP_RATE,URL_CLICK,CLICK_RATE
NAME_SALES_LOCATION_PROVINCE,,,,,,,,,,
Anhui,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-10 10:58:15,10033,1.0,199,0.019835,1194,0.119007,0,0.0
Anhui,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-12 10:40:55,1721,1.0,14,0.008135,107,0.062173,0,0.0
Beijing,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-10 10:58:15,3496,1.0,75,0.021453,385,0.110126,0,0.0
Beijing,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-12 10:40:55,458,1.0,13,0.028384,42,0.091703,0,0.0
Chongqing,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-10 10:58:15,6526,1.0,197,0.030187,1038,0.159056,0,0.0
Chongqing,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-12 10:40:55,434,1.0,8,0.018433,41,0.094470,0,0.0
Fujian,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-10 10:58:15,10330,1.0,350,0.033882,1938,0.187609,0,0.0
Fujian,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-12 10:40:55,1414,1.0,37,0.026167,172,0.121641,0,0.0
Gansu,TAPP_ALDI2_Promo_Tipper_SCR-1382,2020-06-10 10:58:15,4396,1.0,128,0.029117,970,0.220655,0,0.0


In [36]:
SMTByDate[['CAMPAIGN_NM', 'LAUNCH_DATE', 'DELIVERY_OK', 'DELIVERY_RATE', 'INBOX_READ', 'INBOX_RATE', 'POPUP_READ', 'POPUP_RATE', 'URL_CLICK', 'CLICK_RATE']].to_csv('t1')

In [270]:
#数据集组装
df3 = df1.append(df2)
DataSet = pd.concat([df3[cols2[0:6] + cols2[10:14] + cols2[16:]],df['CHECK_MODEL']], axis=1, join_axes=[df3.index])
# DataSet

In [421]:
DataSet.columns
# cols2[0:6] + cols2[10:14] + cols2[16:]

Index(['LAUNCH_DATE', 'CAMPAIGN_CD', 'CAMPAIGN_NM', 'CAMP_FREQUENCY', 'CHANNEL_CD', 'ID_TEMPLATE', 'CAMP_BUSINESS_TARGET', 'CAMP_TYPE', 'CAMP_TARGET_GROUP_TYPE', 'CAMP_BUSINESS_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_OK', 'DELIVERY_FAILED', 'CNT_REPLY', 'CNT_CONTACT_SUCCESS', 'CNT_CONTACT_FAILED', 'INBOX_READ', 'POPUP_READ', 'PUSH_READ', 'RED_PACKET_SUCCESS', 'RED_PACKET_FAILED', 'URL_CLICK', 'CNT_REMOVE', 'CHECK_MODEL'], dtype='object')

In [338]:
# By Data Check
DataSetByDate = DataSet[(DataSet['CHECK_MODEL']=='d')].sort_values(by=['CHANNEL_CD','CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE'])
# By Temp Check
DataSetByTemp = DataSet[(DataSet['CHECK_MODEL']=='l')].sort_values(by=['CHANNEL_CD','CAMPAIGN_NM','LAUNCH_DATE','ID_TEMPLATE'])

In [345]:
# DataSetByDate

In [346]:
CHANNEL_CD = list(DataSetByDate.drop_duplicates(['CHANNEL_CD'])['CHANNEL_CD'])
for i in CHANNEL_CD:
    if len(DataSetByDate.loc[(DataSetByDate['CHANNEL_CD'] == i)])>0:
        print(i)
    if len(DataSetByTemp.loc[(DataSetByTemp['CHANNEL_CD'] == i)])>0:
        print(i)

SMS
SMT
SMT


In [509]:
# SMS By Data
SMSColumns = ['CAMP_TARGET_GROUP_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_OK', 'DELIVERY_FAILED', 'DELIVERY_RATE', 'CNT_REPLY', 'REPLY_RATE']
SMSByDate = DataSetByDate.loc[(DataSetByDate['CHANNEL_CD'] == 'SMS')].copy()
SMSByDate['DELIVERY_RATE'] = SMSByDate.apply(lambda x: x['DELIVERY_OK']/x['TOTAL_DELIVERY'], axis=1)
SMSByDate['REPLY_RATE'] = SMSByDate.apply(lambda x: x['CNT_REPLY']/x['DELIVERY_OK'], axis=1)
SMSByDate = SMSByDate.replace(0,"").set_index(['CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE'])
SMSByDate = SMSByDate[SMSColumns]
SMSByDate.columns=[i.replace('_', ' ') for i in SMSColumns]
SMSByDate
# print(SMSByDate.to_html(columns=SMSColumns, float_format=a, border=1, classes='fl-table'))

CAMP TARGET GROUP TYPE  TOTAL DELIVERY  DELIVERY OK  DELIVERY FAILED  DELIVERY RATE CNT REPLY REPLY RATE
CAMP_FREQUENCY CAMPAIGN_NM                                LAUNCH_DATE                                                                                                         
Weekly         SMS_Performance_Report_L14d_TIPPER_SCR_879 2019-11-26          Tipper               13029         12207           821          0.936910                        
                                                          2019-12-10          Tipper               12289         11365           924          0.924811                        
                                                          2019-12-24          Tipper               12865         12067           796          0.937971

In [499]:
SMSColumns = ['CAMP_TARGET_GROUP_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_OK', 'DELIVERY_FAILED', 'DELIVERY_RATE', 'CNT_REPLY', 'REPLY_RATE']
[i.replace('_', ' ') for i in SMSColumns]

['CAMP TARGET GROUP TYPE',
 'TOTAL DELIVERY',
 'DELIVERY OK',
 'DELIVERY FAILED',
 'DELIVERY RATE',
 'CNT REPLY',
 'REPLY RATE']

In [334]:
# SMSByDate.loc[SMSByDate.index,'INBOX_READ']

In [305]:
# SMT By Data
SMTColumns = ['CAMP_TARGET_GROUP_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'INBOX_READ', 'INBOX_RATE', 'POPUP_READ', 'POPUP_RATE', 'URL_CLICK', 'CLICK_RATE', 'PUSH_READ', 'PUSH_RATE']
SMTByDate = DataSetByDate.loc[(DataSetByDate['CHANNEL_CD'] == 'SMT')].copy()
SMTByDate['DELIVERY_RATE'] = SMTByDate.apply(lambda x: x['DELIVERY_OK']/x['TOTAL_DELIVERY'], axis=1)
SMTByDate['INBOX_RATE'] = SMTByDate.apply(lambda x: x['INBOX_READ']/x['DELIVERY_OK'], axis=1)
SMTByDate['POPUP_RATE'] = SMTByDate.apply(lambda x: x['POPUP_READ']/x['DELIVERY_OK'], axis=1)
SMTByDate['CLICK_RATE'] = SMTByDate.apply(lambda x: x['URL_CLICK']/x['DELIVERY_OK'], axis=1)
SMTByDate['PUSH_RATE'] = SMTByDate.apply(lambda x: x['PUSH_READ']/x['DELIVERY_OK'], axis=1)
SMTByDate = SMTByDate.set_index(['CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE'])
SMTByDate.to_html('test.html', columns=SMTColumns, float_format=a, border=1)

In [ ]:
# GEN By Data
GENColumns = ['CAMP_TARGET_GROUP_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'CNT_CONTACT_SUCCESS', 'CONTACT_SUCCESS_RATE', 'CNT_CONTACT_FAILED', 'CONTACT_FAILED_RATE']
GENByDate = DataSetByDate.loc[(DataSetByDate['CHANNEL_CD'] == 'GEN')].copy()
GENByDate['DELIVERY_RATE'] = GENByDate.apply(lambda x: x['DELIVERY_OK']/x['TOTAL_DELIVERY'], axis=1)
GENByDate['CONTACT_SUCCESS_RATE'] = GENByDate.apply(lambda x: x['CNT_CONTACT_SUCCESS']/x['DELIVERY_OK'], axis=1)
GENByDate['CONTACT_FAILED_RATE'] = GENByDate.apply(lambda x: x['CNT_CONTACT_FAILED']/x['DELIVERY_OK'], axis=1)
GENByDate = GENByDate.set_index(['CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE'])
GENByDate.to_html('test.html', columns=GENColumns, float_format=a, border=1)

In [310]:
# WCH By Data
WCHColumns = ['CAMP_TARGET_GROUP_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'URL_CLICK', 'CLICK_RATE']
WCHByDate = DataSetByDate.loc[(DataSetByDate['CHANNEL_CD'] == 'WCH')].copy()
WCHByDate['DELIVERY_RATE'] = WCHByDate.apply(lambda x: x['DELIVERY_OK']/x['TOTAL_DELIVERY'], axis=1)
WCHByDate['CLICK_RATE'] = WCHByDate.apply(lambda x: x['URL_CLICK']/x['DELIVERY_OK'], axis=1)
WCHByDate = WCHByDate.set_index(['CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE'])
WCHByDate.to_html('test.html', columns=WCHColumns, float_format=a, border=1)

0

In [ ]:
# RPT By Data
RPTColumns = ['CAMP_TARGET_GROUP_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'RED_PACKET_SUCCESS', 'RED_PACKET_SUCCESS_RATE', 'RED_PACKET_FAILED', 'RED_PACKET_FAILED_RATE', 'URL_CLICK', 'CLICK_RATE']
RPTByDate = DataSetByDate.loc[(DataSetByDate['CHANNEL_CD'] == 'RPT')].copy()
RPTByDate['DELIVERY_RATE'] = RPTByDate.apply(lambda x: x['DELIVERY_OK']/x['TOTAL_DELIVERY'], axis=1)
RPTByDate['RED_PACKET_SUCCESS_RATE'] = RPTByDate.apply(lambda x: x['RED_PACKET_SUCCESS']/x['DELIVERY_OK'], axis=1)
RPTByDate['RED_PACKET_FAILED_RATE'] = RPTByDate.apply(lambda x: x['RED_PACKET_FAILED']/x['DELIVERY_OK'], axis=1)
RPTByDate['CLICK_RATE'] = RPTByDate.apply(lambda x: x['URL_CLICK']/x['DELIVERY_OK'], axis=1)
RPTByDate = RPTByDate.set_index(['CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE'])
RPTByDate.to_html('test.html', columns=RPTColumns, float_format=a, border=1)

In [136]:
# By Temp Check
DataSet[(DataSet['CHECK_MODEL']=='l')] 

,LAUNCH_DATE,CAMPAIGN_CD,CAMPAIGN_NM,CHANNEL_CD,CAMP_OWNER,CAMP_FREQUENCY,CAMP_BUSINESS_TARGET,CAMP_TYPE,CAMP_TARGET_GROUP_TYPE,CAMP_BUSINESS_TYPE,...,CNT_CONTACT_SUCCESS,CNT_CONTACT_FAILED,INBOX_READ,POPUP_READ,PUSH_READ,RED_PACKET_SUCCESS,RED_PACKET_FAILED,URL_CLICK,CNT_REMOVE,CHECK_MODEL
CAMP1491,2020-01-07,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,SMT,Leo Zhang,One Time,Sales uplift,POS Loan,Tipper,Tipper,...,0,0,9939,74480,0,0,0,0,0,l
CAMP1491,2020-01-09,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,SMT,Leo Zhang,One Time,Sales uplift,POS Loan,Tipper,Tipper,...,0,0,8635,65951,0,0,0,0,0,l
CAMP1491,2020-01-13,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,SMT,Leo Zhang,One Time,Sales uplift,POS Loan,Tipper,Tipper,...,0,0,6017,34981,0,0,0,0,0,l


In [137]:
# Till Now
DataSet[(DataSet['CHECK_MODEL']=='t')]

,LAUNCH_DATE,CAMPAIGN_CD,CAMPAIGN_NM,CHANNEL_CD,CAMP_OWNER,CAMP_FREQUENCY,CAMP_BUSINESS_TARGET,CAMP_TYPE,CAMP_TARGET_GROUP_TYPE,CAMP_BUSINESS_TYPE,...,CNT_CONTACT_SUCCESS,CNT_CONTACT_FAILED,INBOX_READ,POPUP_READ,PUSH_READ,RED_PACKET_SUCCESS,RED_PACKET_FAILED,URL_CLICK,CNT_REMOVE,CHECK_MODEL


### Class

In [3]:
class CampaingReadRateReports(object):
    def __init__(self, CampaignLaunchDays=12, CampaignEndDays=1, ReponseAnalysisDays=2):
        """
        Init signature: CampaingReadRateReports(CampaignLaunchDays， ReponseAnalysisDays)
        Docstring:
        Check the one-time campaigns in the last 12 days and new regular campaigns recently.
        Daily Campaign : Every template for the first three-time launched and launch time in the last 12 days.
        Weekly Campaign : Every template for the first three-time launched and launch time in the last 28 days.
        Monthly Campaign : Check only the first launch due to data expiration.
        ----------
        CampaignLaunchDays : str, 12(Default)
        ReponseAnalysisDays: str, 5(Default)
        """
        try:
            self.user = conf.read_config("SAS_db", "user")
            self.password = conf.read_config("SAS_db", "password")
            self.service = conf.read_config("SAS_db", "service")
            self.db = cx_Oracle.connect(self.user, self.password, self.service)
            self.con = self.db.cursor()
            self.writer = pd.ExcelWriter(D3_excel)
            self.sender = conf.read_config('E-mail', 'd3_report_to')
            self.copyer = conf.read_config('E-mail', 'copy_to')
            self.CampaignLaunchDays = CampaignLaunchDays
            self.ReponseAnalysisDays = ReponseAnalysisDays
            self.CampaignEndDays = CampaignEndDays
            if self.CampaignLaunchDays == 7 and datetime.datetime.now().strftime("%w") == "1":
                print(ColorMe("Send To Analysis Team").yellow())
                # self.sender = self.sender + "CN.D.SalesAnalyticalTeam-XRMAnalysis@homecredit.cn;"
                # self.copyer = "Umut.Deniz@homecreditcfc.cn;Nancy.Zhang@homecredit.cn;Martin.Tomis@homecredit.cn;" + self.copyer
                print(ColorMe("Sender:").blue(), self.sender)
                print(ColorMe("Copyer:").blue(), self.copyer)
            else:
#                 self.copyer = self.copyer + "Jessie.WangTJ@homecredit.cn;"
                print(ColorMe("Copyer:").blue(), self.copyer)
            print(ColorMe("Success: Initial Complete.").green())
        except Exception as e:
            print(ColorMe("Error: Initial Failed!").red())
            log_info.error(e)
    
    def camp_list(self):
        """
        Docstring:
        To get the campaign list which waits to calculate the reading rate.
        """
        try:
            CampaignListWaitToCheck = os.path.join(BASE_DIR, 'db','sql','CampaignListWaitToCheck.sql')
            with open(CampaignListWaitToCheck, 'r') as f:
                sql = Template(f.read()).substitute(LaunchDays=self.CampaignLaunchDays, EndDays=self.CampaignEndDays)
                
            print(ColorMe("Info: Start to get a check campaign list.").blue())
            row = self.con.execute(sql).fetchall()
            cols = [i[0] for i in self.con.description]
            
            self.df = pd.DataFrame(data=row, index=[i[0] for i in row], columns=cols)
            if len(self.df) > 0:
                self.df.to_excel(excel_writer=self.writer, sheet_name='CampList', index=0)
                self.writer.save()
                self.writer.close()
                print(ColorMe('Success: Campaign List has created successfully - ').green(),os.path.basename(D3_excel))
            else:
                print(ColorMe('Warning: Campaign list is empty, please concern.').yellow())
            # log_info.debug(f'Campaign List({len(list(df["CELL_PACKAGE_SK"]))}):{list(df["CELL_PACKAGE_SK"])}')
            return self.df
        except Exception as e:
            log_info.error(e)
            return pd.DataFrame()
    
    def GetDataSet(self):
        """
        Docstring:
        To get the campaign reading rate results.
        """
        try:
            SubjectTypeList = list(self.df.drop_duplicates(['SUBJECT_TYPE_NM'])['SUBJECT_TYPE_NM'])
            cols = ['LAUNCH_DATE', 'CAMPAIGN_CD', 'CAMPAIGN_NM', 'CAMP_FREQUENCY', 'CHANNEL_CD', 'ID_TEMPLATE', 'CAMP_BUSINESS_TARGET', 'CAMP_TYPE', 'TARGET', 'CAMP_BUSINESS_TYPE', 'TOTAL_DELIVERY', 'DELIVERY_OK', 'DELIVERY_FAILED', 'CNT_REPLY', 'CNT_CONTACT_SUCCESS', 'CNT_CONTACT_FAILED', 'INBOX_READ', 'POPUP_READ', 'PUSH_READ', 'RED_PACKET_SUCCESS', 'RED_PACKET_FAILED', 'URL_CLICK', 'CNT_REMOVE']
            df = pd.DataFrame()
            print(ColorMe("Info: Start to check the campaign reading rate.").blue())
            for i in SubjectTypeList:
                res = self.CheckSubject(i)
                if len(res) == 0:
                    continue
                df = df.append(res)
            self.db.close()
            self.DataSet = pd.concat([df[cols],self.df['CHECK_MODEL']], axis=1, join_axes=[df.index])
            dele = self.DataSet[self.DataSet['TOTAL_DELIVERY']==self.DataSet['CNT_REMOVE']]
            self.DataSet = self.DataSet[self.DataSet['TOTAL_DELIVERY']!=self.DataSet['CNT_REMOVE']].copy()
            cnt = self.DataSet['CAMPAIGN_CD'].value_counts().copy()
            if len(dele)>0:
                self.DataSet.insert(0,'cnt',cnt)
                self.DataSet.loc[self.DataSet[(self.DataSet["cnt"]==1)&(self.DataSet["CHECK_MODEL"]=="l")].index,"CHECK_MODEL"] = "t"
                self.DataSet = self.DataSet.drop(['cnt'],axis=1).copy()
            print(ColorMe('Success: The Check process complete.').green())
            return self.DataSet.sort_values(by=['LAUNCH_DATE'])
        except Exception as e:
            log_info.error(e)
            return pd.DataFrame()
    
    def CheckSubject(self, SubjectType):
        """
        Init signature: CampaingReadRateReports.CheckSubject(SubjectType)
        Docstring:
        Calculate the reading rate by the subject level.
        Parameters
        ----------
        SubjectType : str, 'Employee' or 'Tipper' 
        """
        try:
            SubjectDict = {'Employee': ['CI_CONTACT_HISTORY_EMP', 'CI_RESPONSE_HISTORY_EMP'], 'Tipper': ['CI_CONTACT_HISTORY_TIP', 'CI_RESPONSE_HISTORY_TIP']}
            CellPackageSK = "'"
            CampaignAnaylysis = os.path.join(BASE_DIR, 'db','sql','CampaignAnaylysis.sql')
            print(ColorMe(f"Info: Start to check the {SubjectType} campaign reading rate.").blue())
            for i in self.df[self.df.SUBJECT_TYPE_NM == SubjectType]['CELL_PACKAGE_SK']:
                CellPackageSK = CellPackageSK + i.replace(' ',"','") + "','"
                
            TB1 = SubjectDict[SubjectType][0]
            TB2 = SubjectDict[SubjectType][1]
            SK = CellPackageSK[:-2]
            
            with open(CampaignAnaylysis, 'r') as f:
                sql = Template(f.read()).substitute(TB1=TB1, TB2=TB2, SK=SK, RESDY=self.ReponseAnalysisDays)
                row = self.con.execute(sql).fetchall()
                cols = [i[0] for i in self.con.description]
                
            df = pd.DataFrame(data=row, index=[i[1] for i in row], columns=cols)
            print(ColorMe('Success: Check complete.').green())
            return df
        except Exception as e:
            log_info.error(e)
            return pd.DataFrame()
    
    def TableHtmlCode(self):
        """
        Docstring:
        To Assemble the table HTML code.
        """
        try:
            CheckModelList = list(self.df.drop_duplicates(['CHECK_MODEL'])['CHECK_MODEL'])
            RegularReportHtml = ""
            OneTiemReportHtml = ""
            print(ColorMe(f"Info: Start to assemble the table.").blue())
            if 't' in CheckModelList:
                DataSetByTemp = self.DataSet[(self.DataSet['CHECK_MODEL']=='t')].sort_values(by=['CHANNEL_CD','LAUNCH_DATE','CAMPAIGN_NM'])
                # OneTiemReportHtml = '<h1>Recently added one time campaign reading rate analysis.</h1>'
                OneTiemReportHtml = OneTiemReportHtml + self.TransToHtml('t', DataSetByTemp)

            if 'l' in CheckModelList:
                # if len(OneTiemReportHtml) == 0:
                #     OneTiemReportHtml = '<h1>Recently added one time campaign reading rate analysis.</h1>'

                DataSetByTemp = self.DataSet[(self.DataSet['CHECK_MODEL']=='l')].sort_values(by=['CHANNEL_CD','LAUNCH_DATE','CAMPAIGN_NM','ID_TEMPLATE'])
                OneTiemReportHtml = OneTiemReportHtml + self.TransToHtml('l', DataSetByTemp)

            if 'd' in CheckModelList:
                DataSetByTemp = self.DataSet[(self.DataSet['CHECK_MODEL']=='d')].sort_values(by=['CHANNEL_CD','LAUNCH_DATE','CAMP_FREQUENCY','CAMPAIGN_NM'])
                # RegularReportHtml = RegularReportHtml + '<h1>Recently added regular campaign reading rate analysis.</h1>'
                RegularReportHtml = RegularReportHtml + self.TransToHtml('d', DataSetByTemp)

            ReportHtml = {"RegularReportHtml":RegularReportHtml, "OneTiemReportHtml":OneTiemReportHtml}
            print(ColorMe('Success: Assemble complete.').green())
            return ReportHtml
        except Exception as e:
            log_info.error(e)
            return False
            
    def TransToHtml(self, model, dataframes):
        """
        Init signature: CampaingReadRateReports.TransToHtml(model, dataframes)
        Docstring:
        Transform the DataFrame to HTML by model.
        Parameters
        ----------
        model : str, 'd', 'l' or 't'
            d: Only use in regular Campaing. Index by ['CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE']
            l: Only use in a one-time Campaign with multi-template in the same one. Index by ['CAMPAIGN_NM','ID_TEMPLATE','LAUNCH_DATE']
            t: One-time with single-template in the same one. Index by ['CAMPAIGN_NM','LAUNCH_DATE']
        """
        FloatFormat = lambda x: format(x, '.2%') if x != np.nan else ""
        CHANNEL_CD = list(dataframes.drop_duplicates(['CHANNEL_CD'])['CHANNEL_CD'])
        # start_date = (datetime.datetime.now() - datetime.timedelta(days=12)).strftime("%Y/%m/%d")
        # end_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y/%m/%d")
        CDict = {"SMS":['TARGET', 'TOTAL_DELIVERY', 'DELIVERY_OK', 'DELIVERY_FAILED', 'DELIVERY_RATE', 'CNT_REPLY', 'REPLY_RATE'],
                "SMT":['TARGET', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'INBOX_READ', 'INBOX_RATE', 'POPUP_READ', 'POPUP_RATE', 'URL_CLICK', 'CLICK_RATE'],
                "GEN":['TARGET', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'CNT_CONTACT_SUCCESS', 'CONTACT_SUCCESS_RATE', 'CNT_CONTACT_FAILED', 'CONTACT_FAILED_RATE'],
                "WCH":['TARGET', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'URL_CLICK', 'CLICK_RATE'],
                "RPT":['TARGET', 'TOTAL_DELIVERY', 'DELIVERY_RATE', 'RED_PACKET_SUCCESS', 'RED_PACKET_SUCCESS_RATE', 'RED_PACKET_FAILED', 'RED_PACKET_FAILED_RATE', 'URL_CLICK', 'CLICK_RATE']}
        TableHtml = ""
        for i in CHANNEL_CD:
            print(ColorMe(f"Info: Start to transform the {i} by model-{model}.").blue())
            if len(dataframes.loc[(dataframes['CHANNEL_CD'] == i)]) > 0:
                dframe = dataframes.loc[(dataframes['CHANNEL_CD'] == i)].copy()
                cols = CDict[i]
                dframe['DELIVERY_RATE'] = dframe.apply(lambda x: x['DELIVERY_OK']/x['TOTAL_DELIVERY'], axis=1)
                
                if i == 'SMS':
                    dframe['REPLY_RATE'] = dframe.apply(lambda x: x['CNT_REPLY']/x['DELIVERY_OK'], axis=1)
                elif i == 'SMT':
                    dframe['INBOX_RATE'] = dframe.apply(lambda x: x['INBOX_READ']/x['DELIVERY_OK'], axis=1)
                    dframe['POPUP_RATE'] = dframe.apply(lambda x: x['POPUP_READ']/x['DELIVERY_OK'], axis=1)
                    dframe['CLICK_RATE'] = dframe.apply(lambda x: x['URL_CLICK']/x['DELIVERY_OK'], axis=1)
                    # dframe['PUSH_RATE'] = dframe.apply(lambda x: x['PUSH_READ']/x['DELIVERY_OK'], axis=1)
                elif i == 'GEN':
                    dframe['CONTACT_SUCCESS_RATE'] = dframe.apply(lambda x: x['CNT_CONTACT_SUCCESS']/x['DELIVERY_OK'], axis=1)
                    dframe['CONTACT_FAILED_RATE'] = dframe.apply(lambda x: x['CNT_CONTACT_FAILED']/x['DELIVERY_OK'], axis=1)
                elif i == 'WCH':
                    dframe['CLICK_RATE'] = dframe.apply(lambda x: x['URL_CLICK']/x['DELIVERY_OK'], axis=1)
                elif i == 'RPT':
                    dframe['RED_PACKET_SUCCESS_RATE'] = dframe.apply(lambda x: x['RED_PACKET_SUCCESS']/x['DELIVERY_OK'], axis=1)
                    dframe['RED_PACKET_FAILED_RATE'] = dframe.apply(lambda x: x['RED_PACKET_FAILED']/x['DELIVERY_OK'], axis=1)
                    dframe['CLICK_RATE'] = dframe.apply(lambda x: x['URL_CLICK']/x['DELIVERY_OK'], axis=1)
                else:
                    raise ValueError('Wrong Channel Type!!!')
                
                if model == 'd':
                    dframe = dframe.replace(0,"").set_index(['CHANNEL_CD','CAMP_FREQUENCY','CAMPAIGN_NM','LAUNCH_DATE'])
                    # TableHtml = TableHtml + f'<h2>{i} Channel {start_date} - {end_date} Reading Rate Analysis By Launch Date </h2>'
                elif model == 'l':
                    dframe = dframe.replace(0,"").set_index(['CHANNEL_CD','CAMPAIGN_NM','ID_TEMPLATE','LAUNCH_DATE'])
                    # TableHtml = TableHtml + f'<h2>{i} Channel {start_date} - {end_date} Reading Rate Analysis By Template </h2>'
                elif model == 't':
                    dframe = dframe.replace(0,"").set_index(['CHANNEL_CD','CAMPAIGN_NM','LAUNCH_DATE'])
                    # TableHtml = TableHtml + f'<h2>{i} Channel {start_date} - {end_date} Reading Rate Analysis Till Now</h2>'
                else:
                    raise ValueError('Wrong Check Model Code!')
                dframe = dframe[cols]
                dframe.columns=[i.replace('_', ' ') for i in cols]
                htmlCode = dframe.to_html(columns=[i.replace('_', ' ') for i in cols], float_format=FloatFormat, border=1, classes='fl-table')
                TableHtml = TableHtml + htmlCode
            else:
                print(ColorMe(f"Waring:Channel {i} hasn't data in model-{model}.").yellow())
        
        return TableHtml
    
    def CreateReport(self):
        """
        Docstring:
        To Create the report.
        """
        ReportHtml = self.TableHtmlCode()
        RegularReportHtml = ReportHtml['RegularReportHtml']
        OneTiemReportHtml = ReportHtml['OneTiemReportHtml']
        start_date = (datetime.datetime.now() - datetime.timedelta(days=int(self.CampaignLaunchDays))).strftime("%Y/%m/%d")
        end_date = (datetime.datetime.now() - datetime.timedelta(days=int(self.CampaignEndDays))).strftime("%Y/%m/%d")
        if len(RegularReportHtml)>0:
            with open(MyTemp, 'r') as f:
                content = Template(f.read()).substitute(frequncy='regular', start_date=start_date, end_date=end_date, tableframe=RegularReportHtml).replace('dataframe fl-table','fl-table')
				
                if self.CampaignLaunchDays == 7 and datetime.datetime.now().strftime("%w") == "1":
                    subject = f"XRM Regular Campaign Reading Rate Analysis in last 1 week"
                elif self.CampaignLaunchDays == 12 and datetime.datetime.now().strftime("%w") == "5":
                    subject = f"XRM Regular Campaign Reading Rate Analysis in last 2 weeks"
                else:
                    subject = f"XRM Regular Campaign Reading Rate Analysis in last {str(round(int(self.CampaignLaunchDays)/7))} week"
                
            with open(D3_html_R, 'w') as f:
                f.write(content)
                f.close()
				
            if not self.SentEmail(subject, content):
                return False
            else:
                f.close()

            print(ColorMe('Success: Regular Campaign Reading Rate report generated.').green())
            
                
        if len(OneTiemReportHtml)>0:   
            with open(MyTemp, 'r') as f:
                content = Template(f.read()).substitute(frequncy='one time', start_date=start_date, end_date=end_date, tableframe=OneTiemReportHtml).replace('dataframe fl-table','fl-table')
				
                if self.CampaignLaunchDays == 7 and datetime.datetime.now().strftime("%w") == "1":
                    subject = f"XRM One-Time Campaign Reading Rate Analysis in last 1 week"
                elif self.CampaignLaunchDays == 12 and datetime.datetime.now().strftime("%w") == "5":
                    subject = f"XRM One-Time Campaign Reading Rate Analysis in last 2 week"
                else:
                    subject = f"XRM One-Time Campaign Reading Rate Analysis in last {str(round(int(self.CampaignLaunchDays)/7))} week"
            
            with open(D3_html_O, 'w') as f:
                f.write(content)
                f.close()
             
            if not self.SentEmail(subject, content):
                return False
            else:
                f.close()
            
            print(ColorMe('Success: One time Campaign Reading Rate report generated.').green())
            
        return True
    
    def SentEmail(self, subject, content):
        """
        Docstring:
        To sent the report.
        """
        if not send_email.outlook(send_to=self.sender, subject=subject, content=content, cc_to=self.copyer, addtach=None):
            logging.error('E-mail Send Faild!')
            print(ColorMe(f'{datetime.datetime.now().strftime("%Y_%m_%d %X")} E-mail Send Faild!').red())
            return False
        else:
            logging.info('E-mail Send Success!')
            print(ColorMe(f'{datetime.datetime.now().strftime("%Y_%m_%d %X")} E-mail Send Success!').green())
            return True

## Test

### step test

In [483]:
test = D3_Reports()
camp_list = test.camp_list()
DataSet = test.GetDataSet()

In [485]:
TableHtmlCode = test.TableHtmlCode()

In [446]:
camp_list

,CAMPAIGN_CD,COMMUNICATION_CD,CELL_PACKAGE_SK,CHANNEL_CD,SUBJECT_TYPE_NM,TOTAL_SEND,PROCESSED_DTTM,CAMP_OWNER,CAMP_FREQUENCY,CAMP_BUSINESS_TARGET,CAMP_TYPE,CAMP_TARGET_GROUP_TYPE,CAMP_BUSINESS_TYPE,CAMP_REQ_TICKET_NUMBER,CHECK_MODEL
CAMP1340,CAMP1340,COMM5581,18987 21123 22026,SMS,Tipper,38183,2019-11-26 15:40:58,Jason Ji,Weekly,Sales uplift,POS Loan,Tipper,Tipper,SCR-879,d
CAMP1367,CAMP1367,COMM5691 COMM5873 COMM6069 COMM6291,19250 20585 21256 21707,SMT,Employee,5752,2019-11-28 15:03:43,Iris Ji,Weekly,Sales uplift,POS Loan,DSM,Sales Network,SCR-884,d
CAMP1435,CAMP1435,COMM6028 COMM6211 COMM6395,21197 21585 21963,SMT,Employee,9649,2019-12-11 17:43:14,Leo Zhang,Weekly,Sales uplift,POS Loan,DSM,Sales Network,SCR-930,d
CAMP1466,CAMP1466,COMM6289,22595,SMT,Employee,2836,2020-01-06 16:45:25,Leo Zhang,Daily,Sales uplift,POS Loan,SA,Sales Network,SCR-937,d
CAMP1470,CAMP1470,COMM6309 COMM6310,22596 22640 22673 22824 22865 22914,SMT,Employee,1184,2020-01-06 16:58:10,Iris Ji,Daily,Sales uplift,POS Loan,SA,Sales Network,SCR-929,d
CAMP1491,CAMP1491,COMM6782 COMM6783 COMM6785 COMM6786,22644 22693 22810 22907,SMT,Tipper,2576846,2020-01-07 17:18:49,Leo Zhang,One Time,Sales uplift,POS Loan,Tipper,Tipper,SCR-981,l
CAMP1505,CAMP1505,COMM7035,22889,SMT,Tipper,133,2020-01-15 11:31:54,Jason Ji,One Time,Sales uplift,POS Loan,Tipper,Tipper,SCR-993,t


In [447]:
DataSet

,LAUNCH_DATE,CAMPAIGN_CD,CAMPAIGN_NM,CAMP_FREQUENCY,CHANNEL_CD,ID_TEMPLATE,CAMP_BUSINESS_TARGET,CAMP_TYPE,CAMP_TARGET_GROUP_TYPE,CAMP_BUSINESS_TYPE,TOTAL_DELIVERY,DELIVERY_OK,DELIVERY_FAILED,CNT_REPLY,CNT_CONTACT_SUCCESS,CNT_CONTACT_FAILED,INBOX_READ,POPUP_READ,PUSH_READ,RED_PACKET_SUCCESS,RED_PACKET_FAILED,URL_CLICK,CNT_REMOVE,CHECK_MODEL
CAMP1340,2019-11-26,CAMP1340,SMS_Performance_Report_L14d_TIPPER_SCR_879,Weekly,SMS,None,Sales uplift,POS Loan,Tipper,Tipper,13029,12207,821,0,0,0,0,0,0,0,0,0,0,d
CAMP1340,2019-12-10,CAMP1340,SMS_Performance_Report_L14d_TIPPER_SCR_879,Weekly,SMS,None,Sales uplift,POS Loan,Tipper,Tipper,12289,11365,924,0,0,0,0,0,0,0,0,0,0,d
CAMP1340,2019-12-24,CAMP1340,SMS_Performance_Report_L14d_TIPPER_SCR_879,Weekly,SMS,7910,Sales uplift,POS Loan,Tipper,Tipper,12865,12067,796,0,0,0,0,0,0,0,0,0,0,d
CAMP1491,2020-01-07,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7327,Sales uplift,POS Loan,Tipper,Tipper,643988,643988,0,0,0,0,10666,82844,0,0,0,0,0,l
CAMP1491,2020-01-09,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7326,Sales uplift,POS Loan,Tipper,Tipper,644280,644280,0,0,0,0,9560,75544,0,0,0,0,0,l
CAMP1491,2020-01-13,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7325,Sales uplift,POS Loan,Tipper,Tipper,644531,644531,0,0,0,0,7356,51066,0,0,0,0,0,l
CAMP1491,2020-01-15,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7516,Sales uplift,POS Loan,Tipper,Tipper,644047,644047,0,0,0,0,2940,25026,0,0,0,0,0,l
CAMP1505,2020-01-15,CAMP1505,SMT_Survey_Call_JD_TIPPER_SCR_993,One Time,SMT,7320,Sales uplift,POS Loan,Tipper,Tipper,133,133,0,0,0,0,21,42,0,0,0,0,0,t
CAMP1367,2019-11-28,CAMP1367,SMT_AVS_ALDI_POS_No_Tipper_Reminder_DSM_SCR-88...,Weekly,SMT,None,Sales uplift,POS Loan,DSM,Sales Network,1736,1736,0,0,0,0,1452,1637,0,0,0,0,0,d
CAMP1367,2019-12-05,CAMP1367,SMT_AVS_ALDI_POS_No_Tipper_Reminder_DSM_SCR-88...,Weekly,SMT,None,Sales uplift,POS Loan,DSM,Sales Network,1306,1306,0,0,0,0,1059,1240,0,0,0,0,0,d


In [486]:
print(TableHtmlCode['RegularReportHtml'])

<h1>Recently added regular campaign reading rate analysis.</h1><h2>SMT Channel 2020/01/05 - 2020/01/16 Reading Rate Analysis By Launch Date </h2><table border="1" class="dataframe fl-table">
  <thead>
    <tr style="text-align: center;">
      <th></th>
      <th></th>
      <th></th>
      <th>CAMP_TARGET_GROUP_TYPE</th>
      <th>TOTAL_DELIVERY</th>
      <th>DELIVERY_RATE</th>
      <th>INBOX_READ</th>
      <th>INBOX_RATE</th>
      <th>POPUP_READ</th>
      <th>POPUP_RATE</th>
      <th>URL_CLICK</th>
      <th>CLICK_RATE</th>
      <th>PUSH_READ</th>
      <th>PUSH_RATE</th>
    </tr>
    <tr>
      <th>CAMP_FREQUENCY</th>
      <th>CAMPAIGN_NM</th>
      <th>LAUNCH_DATE</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="7" valign="top">Daily</th>
      <th rowspan="6" valign="top">SMT_Low _Per

In [487]:
test.CreateReport()

2020_01_17 10:49:12 E-mail Send Success!
2020_01_17 10:49:12 E-mail Send Success!


### Main

In [9]:
report = CampaingReadRateReports(28,32,2)

Copyer: Leo.ZhangZS@homecredit.cn;
Success: Initial Complete.


In [10]:
report.camp_list()

Info: Start to get a check campaign list.


,CAMPAIGN_CD,COMMUNICATION_CD,CELL_PACKAGE_SK,CHANNEL_CD,SUBJECT_TYPE_NM,TOTAL_SEND,PROCESSED_DTTM,CAMP_OWNER,CAMP_FREQUENCY,CAMP_BUSINESS_TARGET,CAMP_TYPE,TARGET,CAMP_BUSINESS_TYPE,CAMP_REQ_TICKET_NUMBER,CHECK_MODEL


In [96]:
data = report.GetDataSet()
data

Info: Start to check the campaign reading rate.
Info: Start to check the Tipper campaign reading rate.
Success: Check complete.
Info: Start to check the Employee campaign reading rate.
Success: Check complete.
Success: The Check process complete.


,LAUNCH_DATE,CAMPAIGN_CD,CAMPAIGN_NM,CAMP_FREQUENCY,CHANNEL_CD,ID_TEMPLATE,CAMP_BUSINESS_TARGET,CAMP_TYPE,TARGET,CAMP_BUSINESS_TYPE,TOTAL_DELIVERY,DELIVERY_OK,DELIVERY_FAILED,CNT_REPLY,CNT_CONTACT_SUCCESS,CNT_CONTACT_FAILED,INBOX_READ,POPUP_READ,PUSH_READ,RED_PACKET_SUCCESS,RED_PACKET_FAILED,URL_CLICK,CNT_REMOVE,CHECK_MODEL
CAMP1488,2020-01-03,CAMP1488,SMT_Promo_Red_Packet_SCR-978,One Time,SMT,None,Service campaign,POS Loan,Tipper,Tipper,291,291,0,0,0,0,76,127,0,0,0,0,0,t
CAMP1486,2020-01-03,CAMP1486,SMT_Binding_Unregulated_RA_SCR-980,One Time,SMT,None,Sales uplift,POS Loan,DSM,Sales Network,333,333,0,0,0,0,238,298,0,0,0,0,0,t
CAMP1491,2020-01-07,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7327,Sales uplift,POS Loan,Tipper,Tipper,643988,643988,0,0,0,0,7403,60223,0,0,0,0,0,l
CAMP1491,2020-01-09,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7326,Sales uplift,POS Loan,Tipper,Tipper,644280,644280,0,0,0,0,8445,64267,0,0,0,0,0,l
CAMP1491,2020-01-13,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7325,Sales uplift,POS Loan,Tipper,Tipper,644531,644531,0,0,0,0,7933,53181,0,0,0,0,0,l
CAMP1491,2020-01-15,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7516,Sales uplift,POS Loan,Tipper,Tipper,644047,644047,0,0,0,0,4260,28701,0,0,0,0,0,l
CAMP1505,2020-01-15,CAMP1505,SMT_Survey_Call_JD_TIPPER_SCR_993,One Time,SMT,7320,Sales uplift,POS Loan,Tipper,Tipper,133,133,0,0,0,0,31,66,0,0,0,0,0,t
CAMP1491,2020-01-16,CAMP1491,SMT_Collect_BlessWords_Game_Tipper_SCR-981,One Time,SMT,7515,Sales uplift,POS Loan,Tipper,Tipper,644728,644728,0,0,0,0,5189,27476,0,0,0,2226,0,l
CAMP1507,2020-01-17,CAMP1507,SMT_Overlap_POS_TransToALDI2_4th_DSM_SCR-994,One Time,SMT,7097,Service campaign,POS Loan,DSM,Sales Network,60,60,0,0,0,0,53,53,0,0,0,0,0,t
CAMP1508,2020-01-17,CAMP1508,SMT_Overlap_POS_To_AVS_4th_Batch_DSM_SCR-995,One Time,SMT,7098,Sales uplift,POS Loan,DSM,Sales Network,33,33,0,0,0,0,30,29,0,0,0,0,0,t


In [78]:
data[data.index=="CAMP1586"][["LAUNCH_DATE","CAMPAIGN_NM","TOTAL_DELIVERY",'DELIVERY_OK',"INBOX_READ","POPUP_READ","URL_CLICK"]]

,LAUNCH_DATE,CAMPAIGN_NM,TOTAL_DELIVERY,DELIVERY_OK,INBOX_READ,POPUP_READ,URL_CLICK
CAMP1586,2020-02-14,SMT_Upgrade_Notice(3.2.4)_SA_SCR-1051,10223,10223,0,0,0


## Single